#### Título: SVM
#### Objetivo: Clasificación de correos SPAM

##### Autores: 
##### Carrillo Rodríguez Rafael Alejandro
##### López Martínez Alejandra
##### Medina Barreras Daniel Ivan
##### Ortiz Galván Angel Alberto
##### Sandoval González Aranza Valeria




##### Introducción

Este cuaderno (notebook de python) contiene todo lo necesario para el correcto funcionamiento del programa


En este ejemplo se muestra el uso de Apache Sparks con Python para clasificarCorreos SPAM. Este es un ejercicio tradicional en Inteligencia Artificial y constituye basicamenete un "Hola Mundo" ("Hello World"), como en todos los ambitos de la programación.

Problema: <strong>Clasificar Correos SPAM.</strong>

Este es un algoritmo supervisado en donde se cuenta con una gran cantidad de ejemplos de correos SPAM que ya han sido clasificados. En base a este conocimiento, se realiza un modelo matemático de clasificación que nos permite, dado otra correos determinar que corre es SPAM y cual no.

un correo te ofrece como una serie de diferentes palabras


El conjunto de datos (dataset) es un conjunto de correos propios.

El conjunto MNIST consta de alrededor de poco mas de 100 correos de entrenamiento (training images).

Por medio de los correos entrenamiento, se genera el modelo matemático y se obtienen los parámetros o pesos del modelo.

Por medio de las unos correos de evaluación, se evalua de una manera muy ambigua el funcionamiento del modelo

En resumen:

Se cuenta con un arreglo para las etiquetas de los correos de entrenamiento

Se cuenta con un par de bariables para los correos de evaluación

### Tratamiento de los datos

In [9]:
import glob2

def data(route, name):
    filenames = glob2.glob(route)  # list of all .txt files in the directory

    with open(name, 'w', encoding='utf8') as f:
        for file in filenames:
            with open(file, encoding='utf8', errors='ignore') as infile:
                f.write(infile.read()+'\n')

# We pass the route where the spam emails are and the name of the file
data(route="./Datos/Spam/*.txt", name='spam.txt')
# We pass the route where the normal emails and the file name are
data(route="./Datos/Normal/*.txt", name='normal.txt')

iniciar HDFS ejecutando start-dfs.cmd y start-yarn.cmd en cmd:

%%bash

#creamos la Carpeta FuentesMllib en Hadoop

$hdfs dfs -mkdir /FuentesMllib

#Subir archivos de spam.txt y normal.txt a la carpeta FuentesMllib en hadoop

$hdfs dfs -copyFromLocal spam.txt /FuentesMllib

$hdfs dfs -copyFromLocal normal.txt /FuentesMllib}

Debe verificar que el namenode, datanode, resourcemanager y nodemanager se estén ejecutando. También puede abrir localhost: 8088 y localhost: 9870 en su navegador para monitorear su nuevo y brillante Sistema de archivos distribuidos Hadoop:

### Iniciamos Spark

In [ ]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
sc =SparkContext()

#### Importamos las librerias necesarias para el funcinamiento del programa

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import SVMWithSGD

###### Leer archivos

In [ ]:
spam = sc.textFile("hdfs://localhost:9000/FuentesMllib/spam.txt")
normal = sc.textFile("hdfs://localhost:9000/FuentesMllib/normal.txt")

##### Se crea una instancia HashingTF para asignar un numero hasta 10000 palabras diferentes

In [ ]:
tf = HashingTF(numFeatures = 10000)

###### Se divide los archivo por palabras y a cada palabra se le aplica la transformacion a numero

In [ ]:
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
normalFeatures = normal.map(lambda email: tf.transform(email.split(" ")))

###### En este caso se crean conjuntos de datos con una estructura LabeledPoint
###### Se asigna 1 a spam y 0 a normal
###### Las etiquetas son:
###### SPAM = 1
###### NORMAL = 0

In [ ]:
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = normalFeatures.map(lambda features: LabeledPoint(0, features))

###### Se hace la union de los LabeledPoint

In [ ]:
trainingData = positiveExamples.union(negativeExamples)

In [ ]:
trainingData.cache()

###### Ejecuta algoritmo SVM (Support Vector Machine) con SGD() para hacer el entenamiento 

In [ ]:
model = SVMWithSGD.train(trainingData)

### Probamos para un ejemplo positivio de spam y luego uno negativo

In [ ]:
#Aplicamos la misma trasformacion HashingTF para obtener los Vectores y luego aplicamos el modelo

posTest = tf.transform("Un servicio personal de compras que provee legalmente drogas sin prescripcin mdica desde Canad y el resto del mundo. Haz tu pedido de Valium (Diazepam) y te garantizamos que lo tendrs en tu casa en 7 DAS.".split(" "))

print("Prediction for positive test example: %g" % model.predict(posTest))

In [ ]:
# Test de un correo normal
negTest = tf.transform("Un servicio personal de compras que provee legalmente drogas sin prescripcin mdica desde Canad y el resto del mundo. Haz tu pedido de Valium (Diazepam) y te garantizamos que lo tendrs en tu casa en 7 DAS.".split(" "))

print ("Prediction for negative test examples: %g" % model.predict(negTest))

In [ ]:
sc.stop()